In [1]:
from random import randint, choices
from typing import Tuple
import matplotlib
import sympy as sp
from numpy.random import choice
from sympy import latex
from sympy.abc import x, y, z

In [2]:
matplotlib.use("Agg")

In [3]:
%run document_generator.ipynb

In [4]:
def latex_image(path: str) -> str:
    return r"""\begin{figure}[H]
    \centering
    \includegraphics[width=0.5\textwidth]{%s}
    \label{fig:figure}
\end{figure}""" % path

In [5]:
def random_float_coef():
    return sp.S(choice(list(set(range(-3, 4)) - {0}))) / choice([1, 2, 3, 5])


def generate_random_surface() -> sp.Expr:
    surface = None
    while surface is None or len(surface.free_symbols) < 3:
        surface = sum([random_float_coef() * item
                       for item in choice([x ** 2, y ** 2, z ** 2, x * y, x * z, y * z, x, y, z, 1],
                                          size=choices([3, 4, 5, 6], weights=[2, 6, 1, 1])[0],
                                          replace=False)])
    return surface


def generate_point_on_surface(surface: sp.Expr) -> sp.Point3D:
    random_x = randint(-3, 3)
    random_y = randint(-3, 3)
    surface_eq_in_x_y_point = surface.subs({x: random_x, y: random_y})
    z_in_x_y_point = sp.solveset(sp.Eq(surface_eq_in_x_y_point, 0), z, sp.Reals).args[0]
    return sp.Point3D(random_x, random_y, z_in_x_y_point)

In [6]:
DESCRIPTION = "Дана поверхность. Составить уравнение касательной плоскости и нормали в данной точке. Изобразить на графике поверхность и касательную плоскость"

OUT_ANSWER_PATH = "../out/answers/answer16/"

In [7]:
def task_condition(surface: sp.Expr, point: sp.Point3D) -> str:
    return f"\n\n${latex(surface)} = 0$,\\ \\ \\ точка\\ $({latex(point.x)}, {latex(point.y)}, {latex(point.z)})$"

In [8]:
def solve_task(option: int, surface: sp.Expr, point: sp.Point3D) -> str:
    point_for_subs = {x: point.x, y: point.y, z: point.z}
    surface_dx_in_point, surface_dy_in_point, surface_dz_in_point = [surface
                                                                     .diff(var)
                                                                     .subs(point_for_subs)
                                                                     .simplify()
                                                                     for var in (x, y, z)]
    tangent_plane = (surface_dx_in_point * (x - point.x) +
                     surface_dy_in_point * (y - point.y) +
                     surface_dz_in_point * (z - point.z)).simplify()
    normal_line = sp.Eq(
        (x - point.x) / surface_dx_in_point,
        sp.Eq(
            (y - point.y) / surface_dy_in_point,
            (z - point.z) / surface_dz_in_point,
            evaluate=False),
        evaluate=False)
    graph = sp.plotting.plot3d(sp.solve(surface, z)[0], sp.solve(tangent_plane, z)[0], show=False)
    graph.save(OUT_ANSWER_PATH + str(option))
    return f"\n\nУравнение касательной плоскости:\n\n" \
           f"$${latex(tangent_plane)} = 0$$\n\n" \
           f"Уравнение нормали:\n\n" \
           f"$${latex(normal_line)}$$\n\n" \
           f"{latex_image(OUT_ANSWER_PATH + str(option) + '.png')}"

In [9]:
def generate_task_and_answer(option: int) -> Tuple[str, str]:
    while True:
        try:
            surface = generate_random_surface()
            point = generate_point_on_surface(surface)
            return task_condition(surface, point), solve_task(option, surface, point)
        except IndexError:
            pass
        except Exception as e:
            print(e)

In [10]:
doc = DocumentGenerator(generate_task_and_answer, DESCRIPTION)

In [ ]:
write_tasks_and_solutions(doc, "../out/tasks/task-16.tex", OUT_ANSWER_PATH + "answer-16.tex", 150)